# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
from create_tables import create_database, drop_tables, create_tables

cur, conn = create_database()
    
drop_tables(cur, conn)
create_tables(cur, conn)

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [8]:
song_files = get_files('data/song_data')

In [9]:
filepath = song_files[0]

In [10]:
filepath

'/home/sboakye/Documents/project_template/data/song_data/A/A/A/TRAAAMO128F1481E7F.json'

In [11]:
data = pd.read_json(filepath, lines=True)
data

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARKFYS91187B98E58F,NaN,,NaN,Jeff And Sheri Easter,267.7024,1,SOYMRWW12A6D4FAB14,The Moon And I (Ordinary Day Album Version),0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [12]:
df = data[['song_id', 'title', 'artist_id', 'year', 'duration']]


In [13]:
song_data = df.values[0].tolist()
song_data

['SOYMRWW12A6D4FAB14',
 'The Moon And I (Ordinary Day Album Version)',
 'ARKFYS91187B98E58F',
 0,
 267.7024]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [14]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [15]:
# artist_data = data.copy()
artist_data = data[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artist_data

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARKFYS91187B98E58F,Jeff And Sheri Easter,,NaN,NaN


In [16]:
# artist_data.rename(columns={'artist_name':'name', 'artist_location':'location', 'artist_latitude':'latitude', 'artist_logitude':'logitude'}, inplace=True)

In [17]:
artist_data = artist_data.values[0].tolist()
artist_data

['ARKFYS91187B98E58F', 'Jeff And Sheri Easter', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [18]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [19]:
log_files = get_files('data/log_data')

In [20]:
filepath = log_files[0]

In [21]:
filepath

'/home/sboakye/Documents/project_template/data/log_data/2018/11/2018-11-20-events.json'

In [22]:
df = pd.read_json(filepath, lines=True)

In [23]:
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Killers,Logged In,Jayden,M,32,Graves,246.80444,paid,"Marinette, WI-MI",PUT,NextSong,1.540664e+12,594,Read My Mind,200,1542672042796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",25
1,Tamia,Logged In,Jayden,M,33,Graves,243.09506,paid,"Marinette, WI-MI",PUT,NextSong,1.540664e+12,594,Officially Missing You (Radio Version),200,1542672288796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",25


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [24]:
df = df[df.page == 'NextSong']
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Killers,Logged In,Jayden,M,32,Graves,246.80444,paid,"Marinette, WI-MI",PUT,NextSong,1.540664e+12,594,Read My Mind,200,1542672042796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",25


In [25]:
df['ts'] = pd.to_datetime(df['ts'], unit='ms')

In [26]:
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Killers,Logged In,Jayden,M,32,Graves,246.80444,paid,"Marinette, WI-MI",PUT,NextSong,1.540664e+12,594,Read My Mind,200,2018-11-20 00:00:42.796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",25


In [27]:
# timestamp, hour, day, week of year, month, year, and weekday 
time_data = []

time_data.append(df.ts.tolist())
time_data.append(df.ts.dt.day.tolist())
time_data.append(df.ts.dt.weekofyear.tolist())
time_data.append(df.ts.dt.month.tolist())
time_data.append(df.ts.dt.year.tolist())
time_data.append(df.ts.dt.weekday.tolist())

In [28]:
# timestamp, hour, day, week of year, month, year, and weekday 
# time_data = ()
column_labels = ['timestamp', 'hour', 'day', 'week_of_year',
                'month', 'year', 'weekday']

In [29]:
dict_data = dict(zip(column_labels, time_data))

In [30]:
time_df = pd.DataFrame(dict_data)
time_df.head()

,timestamp,hour,day,week_of_year,month,year
0,2018-11-20 00:00:42.796,20,47,11,2018,1
1,2018-11-20 00:04:48.796,20,47,11,2018,1
2,2018-11-20 00:08:51.796,20,47,11,2018,1
3,2018-11-20 00:13:21.796,20,47,11,2018,1
4,2018-11-20 00:15:35.796,20,47,11,2018,1


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [31]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [32]:
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Killers,Logged In,Jayden,M,32,Graves,246.80444,paid,"Marinette, WI-MI",PUT,NextSong,1.540664e+12,594,Read My Mind,200,2018-11-20 00:00:42.796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",25


In [33]:
log_data = df

In [34]:
user_df = log_data[['userId', 'firstName', 'lastName', 'gender', 'level']]

In [39]:
user_df.drop_duplicates(inplace=True)

/home/sboakye/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
user_df.head()

,userId,firstName,lastName,gender,level
0,25,Jayden,Graves,M,paid
10,51,Maia,Burke,F,free
19,94,Noah,Chavez,M,free
31,85,Kinsley,Young,F,paid
38,32,Lily,Burns,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [36]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [37]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [38]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.